In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as Func
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.bn = nn.BatchNorm2d(20)
    def forward(self, x):
        x = Func.max_pool2d(self.conv1(x), 2)
        x = Func.relu(x) + Func.relu(-x)
        x = Func.relu(Func.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = self.bn(x)
        x = x.view(-1, 320)
        x = Func.relu(self.fc1(x))
        x = Func.dropout(x, training = self.training)
        x = self.fc2(x)
        x = Func.softmax(x, dim = 1)
        return x

In [28]:
input = torch.rand(32, 1, 28, 28)
model = Net()
with SummaryWriter(log_dir = 'logs', comment = 'Net') as w:
    w.add_graph(model, (input, ))